In [114]:
import pandas as pd

In [115]:
df = pd.read_csv('inadvance_synth.csv', sep=';', index_col=0)
df.head(3)

,age,barthel,charlson,codidiagingreso,codservicioreal,creatinina,drg,estancias,glucosa,hematocrito,...,metastatic_tumor,num_grupoact3_HOSP,numurgenciasprevias,potasio,proteina_c_reactiva,rdw_cv,rdw_sd,sodio,urea,label
0,80,NaN,4.0,NaN,HNEM,0.42,141.0,7.0,110.0,29.548684,...,0.0,NaN,6.0,4.653334,NaN,24.9,83.1,145.0,NaN,0.0
1,67,NaN,5.0,NaN,HNEF,NaN,444.0,11.0,NaN,30.648684,...,0.0,NaN,1.0,NaN,NaN,14.8,46.3,NaN,NaN,0.0
2,75,NaN,13.0,M97303,HHEM,0.92,691.0,14.0,NaN,33.848684,...,0.0,NaN,16.0,5.153334,1.090658,23.1,57.2,135.0,NaN,0.0


In [116]:
# 1. What is the size of the dataframe?
df.shape

(38416, 21)

In [117]:
# 2. What is the mean age?
df['age'].mean()

79.4033215326947

In [118]:
# 3. What is the age standard deviation (std)?
df['age'].std()

8.361737254115944

In [119]:
# 4. Which is the variable with the most amount of missing values? Can you list the name of the variables,
# sorting them by number of missing values?
df.isnull().sum().sort_values(ascending=False)

barthel                28198
num_grupoact3_HOSP     21001
proteina_c_reactiva    18175
urea                   14260
codidiagingreso        11378
glucosa                 9228
potasio                 8646
rdw_sd                  8334
rdw_cv                  8334
hematocrito             8302
leucocitos              8302
sodio                   8142
creatinina              7951
drg                     3640
numurgenciasprevias      552
metastatic_tumor          77
charlson                  77
age                        0
estancias                  0
codservicioreal            0
label                      0
dtype: int64

In [120]:
# 5. Name which are the categorical variables.
df.dtypes[df.dtypes == 'object']

codidiagingreso    object
codservicioreal    object
dtype: object

In [121]:
# 6. Extract the ‘label’ column to another variable. How many positive cases there are? And negatives?
label = df['label']
label.value_counts()

label
0.0    24985
1.0    13431
Name: count, dtype: int64

In [122]:
#▪ Split the dataset in two: train (80%) and test (20%). Use a seed to allow replication.
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)
y_train = train["label"]
train.drop("label", axis=1, inplace=True)
y_test = test["label"]
test.drop("label", axis=1, inplace=True)


In [123]:
y_train.shape, train.shape, y_test.shape, test.shape

((30732,), (30732, 20), (7684,), (7684, 20))

In [124]:
non_num_col = df.select_dtypes(exclude=['number']).columns.tolist()
# df[non_num_col[-1]].value_counts()
df[non_num_col[0]].value_counts()

codidiagingreso
401.9     2734
518.81     643
486        533
250.00     532
414.9      511
          ... 
376.11       1
281.2        1
453.6        1
482.49       1
147.1        1
Name: count, Length: 2038, dtype: int64

In [125]:
# 1. How many samples have each set after the split?
print(train.shape)
print(test.shape)

(30732, 20)
(7684, 20)


In [126]:
df.dtypes[df.dtypes == 'object'].index.tolist()

['codidiagingreso', 'codservicioreal']

### tu jest bład 

In [127]:
from sklearn.preprocessing import OneHotEncoder

categorical_cols_train = train.select_dtypes(include=['object']).columns.tolist()

encoder = OneHotEncoder(handle_unknown='ignore')

train_encoded = encoder.fit_transform(train[categorical_cols_train]).toarray()
# print(f'{train_encoded.shape=}')
test_encoded = encoder.transform(test[categorical_cols_train]).toarray()

train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out(categorical_cols_train))
test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out(categorical_cols_train))

#TU JEST błąd 
print(f'{train_encoded_df.shape=}')
print(f'{train.shape=}')
train_df_encoded = pd.concat([train, train_encoded_df], axis=1, ignore_index=True)
print(f'{train_df_encoded.shape=}')
test_df_encoded = pd.concat([test, test_encoded_df], axis=1, ignore_index=True)

train_df_encoded.drop(categorical_cols_train, axis=1, inplace=True)
test_df_encoded.drop(categorical_cols_train, axis=1, inplace=True)
print(f'{train_df_encoded.shape=}')

train_df_encoded.shape
# 30732

train_encoded_df=       codidiagingreso_003.0  codidiagingreso_003.1  codidiagingreso_005.3   
0                        0.0                    0.0                    0.0  \
1                        0.0                    0.0                    0.0   
2                        0.0                    0.0                    0.0   
3                        0.0                    0.0                    0.0   
4                        0.0                    0.0                    0.0   
...                      ...                    ...                    ...   
30727                    0.0                    0.0                    0.0   
30728                    0.0                    0.0                    0.0   
30729                    0.0                    0.0                    0.0   
30730                    0.0                    0.0                    0.0   
30731                    0.0                    0.0                    0.0   

       codidiagingreso_007.8  codidiagingreso_

: 

Implement the method to deal with categorical variables. Briefly explain the chosen alternative. What
would happen if a variable in the test set contains a category that doesn’t exist on the train set? How
would you deal with this situation?

- Label Encoding involves converting each category into a unique numerical value. For example, if a categorical variable has three categories: 'A', 'B', and 'C', Label Encoding might convert them to 0, 1, and 2 respectively. If a new category is encountered in the test set that was not present in the training set, Label Encoding would still assign a unique numerical value to that category. However, this approach might not be ideal because it implies an ordinal relationship between categories which might not exist.
- One-Hot Encoding creates binary columns for each category and indicates the presence of the category with a 1 or 0. For example, the categories 'Red', 'Green', and 'Blue' would be transformed into three columns: 'Is_Red', 'Is_Green', and 'Is_Blue. If a new category appears in the test set, it would result in a column of all zeros in the one-hot encoded representation
- Binary Encoding first converts categories into numerical values using Label Encoding and then those numerical values are converted into binary code. The binary values are split into separate columns. For example, the numerical value 2 (binary: 010) would result in columns 'Is_0', 'Is_1', and 'Is_0'.Similar to One-Hot Encoding, if a new category appears in the test set, it would result in columns of zeros


Yes i did, thats why i used handle_unknown='ignore' in OneHotEncoder. 


In [ ]:
# Implement a method to deal with missing values using simple techniques such as the imputation by
# the mean or the median
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
imputer.fit(train)
train_trans = imputer.transform(train)
train_trans = pd.DataFrame(train_trans, columns=train.columns)
train_trans.isnull().sum().sort_values(ascending=False)

age                       0
codidiagingreso_716.66    0
codidiagingreso_724.02    0
codidiagingreso_724.01    0
codidiagingreso_723.1     0
                         ..
codidiagingreso_410.61    0
codidiagingreso_410.51    0
codidiagingreso_410.41    0
codidiagingreso_410.31    0
codservicioreal_nan       0
Length: 1944, dtype: int64

In [ ]:
# ▪ The transformation applied on the training set of the data should be the same applied to the test set. 
test_trans = imputer.transform(test)
test_trans = pd.DataFrame(test_trans, columns=test.columns)
train_trans.isnull().sum().sort_values(ascending=False)

age                       0
codidiagingreso_716.66    0
codidiagingreso_724.02    0
codidiagingreso_724.01    0
codidiagingreso_723.1     0
                         ..
codidiagingreso_410.61    0
codidiagingreso_410.51    0
codidiagingreso_410.41    0
codidiagingreso_410.31    0
codservicioreal_nan       0
Length: 1944, dtype: int64

Implement the method to deal with the missing values. Briefly explain the chosen alternative. What
would happen if a variable without missing on the train set appears to have been missing in the test
set? How would you deal with them?
- removing entire rows that contain missing values "train.dropna(inplace=True)"
- Fill missing values with the mean, median, or mode of the column. Mean is useful for continuous variables, while median is less sensitive to outliers. Mode is used for categorical variables.
- Use more sophisticated imputation methods like K-Nearest Neighbors (KNN) or regression models to predict missing values based on other features
- Introduce an additional binary feature indicating whether the value was missing. This way, the model can learn if missingness is informative.

Handling Missing Values in Test Set:

If a variable without missing values in the training set has missing values in the test set, it should be handle by using the same strategy that was applied to the training set:



What if we used the whole dataset to fit the imputer and the categorical variables’ method so we don’t have to worry about unconsidered values? Explain briefly why this is a terrible idea. 

Including the test set during preprocessing can lead to data leakage, where information from the test set is used to influence the preprocessing decisions. Its my lead to overly optimistic performance estimates. Whatsmore then its more difficult to check if your model is not overfited whats mean it has problem wih generalization and its unable to handle new values in real-world scenarios.

In [ ]:
# 1. Import the RandomForestClassifier and create the model with the default arguments
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [ ]:
train_trans.shape, y_test.shape

((38097, 1944), (7684,))

In [ ]:
# 2. Fit the model using the train set and the train labels
rf.fit(train_trans, y_test)

ValueError: Found input variables with inconsistent numbers of samples: [38097, 7684]

In [ ]:
# 3. Get the probabilities for the positive class from the test set
